- enviroment


In [115]:
import gensim
from scipy import spatial
import json
import numpy as np
import nltk
from gensim.models import Word2Vec, Doc2Vec
from gensim import models
import string
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from collections import defaultdict
from math import log
from textblob import TextBlob
from collections import Counter
from nltk.tag import StanfordNERTagger

%matplotlib inline

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import urllib
import sys
import os
import zipfile
import tarfile
 
import hashlib
import re
import itertools
from sklearn import metrics



- data file

In [113]:
train_set = "/Users/alfredchen/Documents/GitHub/query-system/data/training.json"
dev_set = "/Users/alfredchen/Documents/GitHub/query-system/data/devel.json"
test_set = "/Users/alfredchen/Documents/GitHub/query-system/data/testing.json"
doc = "/Users/alfredchen/Documents/GitHub/query-system/data/documents.json"

#save word embeddings
embeddings = "/Users/alfredchen/Documents/GitHub/query-system/models/mymodel-size"

# save qa log
log_file = "/Users/alfredchen/Documents/GitHub/query-system/data/log.txt"
answer_type = "/Users/alfredchen/Documents/GitHub/query-system/data/answer_type_label.txt"
answer_file = "/Users/alfredchen/Documents/GitHub/query-system/data/answer.txt"
ner_jar = "/Users/alfredchen/Documents/GitHub/query-system/data/stanford-ner.jar"
ner_model = "/Users/alfredchen/Documents/GitHub/query-system/data/english.muc.7class.distsim.crf.ser.gz"

- preprocess corpus
    - remove stopwords
    - lemmatize
    - lower case
    - creat paragraph_index
    - create index2paragraph

In [3]:
    def get_stopwords():
        
        words = list(nltk.corpus.stopwords.words())
        
        #some stopwords are helpful in targeting answer type, shall not be removed
        words.remove('what')
        words.remove('where')
        words.remove('when')
        words.remove('who')
        words.remove('how')
        words.remove('which')
        # add more #

        stopwords = {}
        for word in words:
            stopwords[word] = stopwords.get(word,0) + 1
        return stopwords

In [4]:
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word, 'n')
    return lemma

def removePunc(string):
    # define punctuation
    punctuations = '''!()-[]{};:'"\,./?@#$%^&*_~'''
    
    # remove punctuation from the string
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    #print(no_punct)
    return no_punct


stopwords = get_stopwords()        # get stopwords

def preprocess_docs(corpus):
        para2index = {}
        index2para = {}
        new_corpus = []
        para_len = 0
        para_count = 0
        for _id, doc in enumerate(corpus):
            new_doc = []
            for _para,para in enumerate(doc['text']):
                para2index[para] = (_id,_para)         # para index
                index2para[(_id,_para)] = para         # doc_id, answer_para index
                new_para = []
                sents = para.split('.')
                para_count += 1
                for _sent,sent in enumerate(sents):
                    new_sent=[]
                    sent = removePunc(sent)
                    #sent = '<s> ' + sent + ' <end>'  # padding
                    words = sent.split(' ')
                    s = ''.join(sent)
                    if s == '<s> <end>':
                        continue
                    for word in words:
                        word = word.strip(',')
                        word = word.strip(',')
                        word = word.strip('.')
                        word = word.strip('?')
                        word = word.strip('\'s')
                        word = word.strip('\"')
                            #if not word.isalpha():
                                #continue
                        new_word = word.lower()
                        new_word = lemmatize(new_word)
                        if stopwords.get(new_word):
                            continue
                        if new_word == '':
                            continue
                        para_len += 1
                        new_sent.append(new_word)                
                    new_para.append(new_sent)
                new_doc.append(new_para)
            new_corpus.append(new_doc)
        average_para_len = (para_len/para_count)
        print(average_para_len)
        return new_corpus, para2index, index2para

In [5]:
docs = json.load(open(doc))
corpus,para2index,index2para = preprocess_docs(docs)

71.1837880794702


- get word embeddings based on pre processed corpus

In [6]:
    def w2v(corpora,size,iter):
        docs = corpora
        sentences = []
        for doc in docs:
            for sents in doc:
                new_para = []
                for sent in sents:
                    new_para += sent
                sentences.append(new_para)
        model = Word2Vec(sentences, size=size, window=5,iter=iter,workers=4)
        model.save(embeddings+str(size)+'-iter'+str(iter))
        return model

In [7]:
#model = w2v(corpus,160,150)   # train the model

#load pre-trained model
model = gensim.models.Word2Vec.load(embeddings+str(160)+'-iter'+str(150))

In [8]:
def term_freqs(corpus):
    tfs = defaultdict(dict)
    tfs_forward = defaultdict(dict)
    doc_id = 0
    for doc in corpus:
        for para in doc:
            for sent in para:
                for term in sent:
                    tfs[term][doc_id] = tfs[term].get(doc_id, 0) + 1 
                    tfs_forward[doc_id][term] = tfs[doc_id].get(term, 0) + 1
        doc_id += 1
    return tfs,doc_id+1,tfs_forward


tfs,total_docment,tfs_forward = term_freqs(corpus)

def get_tfidf(tfs, total_docment,tfs_forward):
    document_length = {}
    for doc_id,doc_list in tfs_forward.items():
        length = 0
        for term, freq in doc_list.items():
            length += freq ** 2
        length = length **0.5
        document_length[doc_id] = length
    tfidf = defaultdict(dict)
    for term, doc_list in tfs.items():
        df = len(doc_list)
        for doc_id, freq in doc_list.items(): 
            tfidf[term][doc_id] = (float(tfs[term][doc_id]) * log(total_docment / df)) / document_length.get(doc_id)
    return tfidf

tfidf = get_tfidf(tfs, total_docment,tfs_forward)






- transform sentence to feature vector (sentence embeddings)

In [9]:
    index2word_set = set(model.wv.index2word)
    
    def get_weight(word,doc_id):
        try:
            weight = tfidf[word][doc_id]
        except KeyError:
            weight = 0.0001
        return weight 
    
    def sent2vec(sentence, model, size, index2word_set, doc_id):
        """transform word embeddings to sentence vector
        param: sentence: sentence that want to be transformed
        param: model: pre-trained word embeddings
        param: size: feature vector dimension
        param: index2word_set
        return: transformed sentence vector
        """        
        try:
            words = sentence.split()  
        except:
            words = sentence
            
        words = [removePunc(x) for x in words]
        feature_vec = np.zeros((size,), dtype='float32')
        n_words = 0
        for word in words:
            word = word.lower()
            word = lemmatize(word)
            if stopwords.get(word):
                continue
            if word in index2word_set:
                n_words += 1
                if doc_id != 'disable':
                    w = get_weight(word,doc_id)
                else:
                    w = 1
                feature_vec = np.add(feature_vec, model.wv[word]*w)
        if (n_words > 0):
            feature_vec = np.divide(feature_vec, n_words)
        return feature_vec

In [10]:
print(get_weight('achim',1))

        


0.18433142253254897


- play with sentence vector

In [11]:
# calculate similarity of two sentence

s1_afv = sent2vec('A kilogram could be definined as having a Planck constant of what value?', model, 160, index2word_set,'disable')
s2_afv = sent2vec('Possible new definitions include "the mass of a body at rest whose equivalent energy equals the energy of photons whose frequencies sum to 7050135639273999999♠135639274×1042 Hz", or simply "the kilogram is defined so that the Planck constant equals 6966662606895999999♠6.62606896×10−34 J⋅s".', model, 160, index2word_set,'disable')
sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
print(sim)
model.wv.most_similar_cosmul(positive=['male', 'queen'], negative=['female'])


0.6428887844085693


[('king', 0.7751615643501282),
 ('balmoral', 0.748808741569519),
 ('duke', 0.7470282912254333),
 ('fief', 0.7448990941047668),
 ('royal', 0.7361407279968262),
 ('lambert', 0.7349075078964233),
 ('controversially', 0.7227569818496704),
 ('prince', 0.7206754088401794),
 ('frederick', 0.7180166244506836),
 ('elizabeth', 0.7173427939414978)]

- get similarity ranking of query to paragraphs in a documents

In [12]:
def sim_sent(query, corpus):
    sim2ipara = {}
    index2sim = {}
    for _id,doc in enumerate(corpus):
        q = sent2vec(query, model, 160, index2word_set,_id)
        for _para,para in enumerate(doc):
            sentence = []
            for sent in para:
                sentence += sent
            sentence = sent2vec(sentence, model, 160, index2word_set,'disable')
            sim = 1 - spatial.distance.cosine(q, sentence)
            index2sim[_id, _para] = sim
            sim2ipara[sim] = (_id,_para)
    return index2sim, sim2ipara
    

#index2sim, sim2ipara = sim_sent("what does the Planck constant refer to?",corpus)



In [13]:
def sentence_similarity(query, answer):
    sim = 1 - spatial.distance.cosine(query, answer)
    return sim
    

In [17]:
from nltk.tokenize import word_tokenize

def getTopN(N,query):
    top_document_id = Counter()
    for token in word_tokenize(query):
        token = token.lower()
        token = lemmatize(token)
        if not stopwords.get(token):
            term_tfidf = tfidf[token]
            for docid, weight in term_tfidf.items():
                 top_document_id[docid] += weight
    top_document_id = top_document_id.most_common(N)
    top_document = []
    for document_id,weight in top_document_id:
        top_document.append(document_id)
    sim2index = {}
    q = sent2vec(query, model, 160, index2word_set,'disable')
    for _id in top_document:
        #q = sent2vec(query, model, 160, index2word_set,_id)
        paras = [para for (doc,para) in index2para.keys() if doc == _id]
        for _para in paras:
            answer = index2para.get((_id,_para))
            answer = sent2vec(answer, model, 160, index2word_set,'disable')
            sim = sentence_similarity(q, answer)
            sim2index[sim] = (_id,_para)
    
    similarity_ranks = sorted(sim2index.keys(),reverse=True)   #sims
    rank = []
    for i in range(N):
        try:
            rank.append(sim2index.get(similarity_ranks[i]))
        except IndexError as e:
            print(top_document_id)
    logs_file = open(log_file,"a")
    logs_file.write(str(rank)+'\n')
    logs_file.close()
    return rank
    
        

index = getTopN(10,
                "A kilogram could be definined as having a Planck constant of what value?")    
print(index)

    

[(0, 5), (0, 1), (0, 16), (0, 13), (0, 23), (0, 11), (0, 17), (0, 2), (0, 10), (0, 0)]


- get top n answer paragraph given quey

In [18]:
"""
def getTopN(N,query):
    index2sim, sim2ipara = sim_sent(query,corpus)
    
    similarity_ranks = sorted(index2sim.values(),reverse=True)   #sims
    rank = []
    for i in range(N):
        rank.append(sim2ipara.get(similarity_ranks[i])) 
        
    return rank
    

getTopN(10,"Who developed the theory of inheritance known as pangenesis?")
"""
pass

- test hit accuracy: return top 10 paragraphs, is given answer included?

In [19]:
def gold_standard(dataset):
    file = json.load(open(dataset))
    gold2index={}
    for line in file:
        _id = line['docid']
        try:
            _para = line['answer_paragraph']
        except KeyError:
            _para = line['id']
        gold2index[line['question']]=(_id,_para)
    return gold2index

def my_qa(n,dataset):
    file = json.load(open(dataset))
    qa2index={}
    for i in range(n):
        save = open(log_file,"a")
        line = file[i]
        query = line['question']
        possible = getTopN(10,query)
        qa2index[line['question']] = possible
        record = line['question']+':'+str(possible)+'\n'
        save.write(record)
        save.close()
    return qa2index
    
    

In [20]:

def acc(gold_standard,qas):
    query = qas.keys()
    total = 0
    correct = 0
    for i,q in enumerate(query):
        gold = gold_standard.get(q)
        if gold in qas.get(q):
            correct += 1
        total += 1
    acc = correct/total
    return acc



In [419]:
"""
#test on top 100 trainset
gold_index = gold_standard(train_set)
qa_index = my_qa(100,train_set)
accu = acc(gold_index,qa_index)
print(accu)
"""
pass

In [417]:


#test on entire devtset
gold_index = gold_standard(dev_set)
qa_index = my_qa(100,dev_set)
acc = acc(gold_index,qa_index)
print(acc)


0.66


- prepare relevance training data
assign label 1 denote relevance, label0 denote irrelevance
randomly choose incorrect query-answer pairs in trainset and labeled 0
choose top N correct query-answer pairs in trainset and labeled 1

In [22]:
model.most_similar(['movie'], topn=10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('film', 0.6140520572662354),
 ('spielberg', 0.43539032340049744),
 ('console', 0.4009501338005066),
 ('sylvester', 0.3827870786190033),
 ('animate', 0.3810267746448517),
 ('tv', 0.37869757413864136),
 ('entertainment', 0.37179452180862427),
 ('stallone', 0.36797311902046204),
 ('tudio', 0.3666054904460907),
 ('television', 0.3636029362678528)]

In [ ]:
def prepare_relevance_data(N):
    file = json.load(open(train_set))
    query = [x['question'] for x in file]
    answer = [x['text'] for x in file]

    pair = []
    for i in range(N):
        q = query[i]
        a = answer[i]
        pair.append(((q,a),1))
        random_q = random_a = [i]
    
        while random_q[0] == i:
            random_q = np.random.choice(N,1)
        while random_a[0] == i or random_a[0] == random_q[0]:
            random_a = np.random.choice(N,1)
        q = query[random_q[0]]  
        a = answer[random_a[0]]
        pair.append(((q,a),0))
    return pair

trainset = prepare_relevance_data(40000)
devset = prepare_relevance_data(3097)
    


    
    

In [31]:
def transform_query(query,answer):
    q = [x.lower().strip('?') for x in query.split()]
    trans_form_query = []
    for word in q:
        if word == 'what' or \
        word == 'who' or \
        word == 'where' or \
        word == 'when' or \
        word == 'which'or \
        word == 'how'or \
        word == 'whose':
            word = str(answer)
        trans_form_query.append(word)
    q = ' '.join(trans_form_query)
    return q    

In [39]:
# relation extraction 
def extract_nouns(context):
    constituents = []
    blob = TextBlob(context)
    constituents = blob.noun_phrases
    return constituents

def get_para_constituent(topk):
    constituents = []
    for index in topk:
        context = index2para.get(index)      
        constituents += extract_nouns(context)
    return constituents
    

        
    
    

    

In [40]:
#just a demo, need more pre-extracted potential answer sets

import ast
def prepare_train(N,dataset):
    file = json.load(open(dataset))
    f = open(log_file,"r")
    corrects = []
    wrongs = []
    for i,tops in enumerate(f):
        line = file[i]
        query = line['question']
        answer = line['text']
        correct = transform_query(query,answer)
        corrects.append(correct)
        topk = ast.literal_eval(tops.strip())
        constituents = get_para_constituent(topk)
        for n in range(5):
            cons = np.random.choice(constituents,1)
            wrong = transform_query(query, cons[0])
            wrongs.append(wrong)
    return corrects, wrongs

correct, wrong = prepare_train(4155,train_set)
label = []

for i in correct:
    label.append(1)
for i in wrong:
    label.append(0)
    
print("done")
            
    

done


In [57]:
print(len(correct))
print(len(wrong))

4741
23705


In [50]:
def prepare_feature_vector(correct,wrong):
    matrix = []
    for i in correct:
        vector = sent2vec(i, model, 160, index2word_set,'disable')
        matrix.append(vector)
    for i in wrong:
        vector = sent2vec(i, model, 160, index2word_set,'disable')
        matrix.append(vector)
    return matrix

X = prepare_feature_vector(correct,wrong)
print("done")


done


In [52]:
print(len(X))
print(len(label))

28446
28446


- binary task, long feature vector -> take SVM classifier

In [56]:
def train(X,y):   
    model = LogisticRegression()
    best_model = model.fit(X, y)
    return best_model
            
relevance_model = train(X,label)
print("done")

done


In [44]:
def pred(query,entity,classifier):
    
    q = transform_query(query,entity)
    vector = sent2vec(q, model, 160, index2word_set,'disable')
    preds = classifier.predict([vector][0:])
    return preds
    
pred = pred("What example is given as another paired relationship of uncertainly related to standard deviation?",
            "time vs. energy",
           relevance_model)

In [58]:
q = []
l = []
for i in range(4741):
    q.append(sent2vec(wrong[i], model, 160, index2word_set,'disable'))
    l.append(0)
for i in correct:
    q.append(sent2vec(i, model, 160, index2word_set,'disable'))
    l.append(1)
    
preds = relevance_model.predict(q[0:])
    
    

# from sklearn.externals import joblib

#save clf
joblib.dump(relevance_model, '/Users/alfredchen/Documents/GitHub/query-system/models/relevance.pkl')

#load clf
#relevance_model = joblib.load('/Users/alfredchen/Documents/GitHub/query-system/models/relevance.pkl') 

In [214]:
def relevant(query, answer, classifier):
    X = []
    for i in range(len(query)):
        q = transform_query(query[i],answer[i])
          
        x = sent2vec(q, model, 160, index2word_set,'disable')
        #x = np.concatenate((q,a),axis=0)
        X.append(x)
    preds=relevance_model.predict(X[0:])
    return preds

#dev set

query=[]
answer = []
labels = []
for pair,label in devset:
    query.append(pair[0])
    answer.append(pair[1])
    labels.append(label)
    
preds = relevant(query,answer,relevance_model)
    

In [59]:
def print_metrices(y_test,y_pred):
    print("Accuracy:")
    print(metrics.accuracy_score(y_test,y_pred))

    print("\nAverage precision:")
    print(metrics.precision_score(y_test,y_pred,average='macro'))

    print("\nAverage recall:")
    print(metrics.recall_score(y_test,y_pred,average='macro'))
    
    print("\nAverage f1:")
    print(metrics.f1_score(y_test,y_pred,average='macro'))

    print("\nClassification report:")
    print(metrics.classification_report(y_test, y_pred))
    
print_metrices(l, preds)

Accuracy:
0.5046403712296984

Average precision:
0.6785870830765016

Average recall:
0.5046403712296984

Average f1:
0.3451919622241372

Classification report:
             precision    recall  f1-score   support

          0       0.50      1.00      0.67      4741
          1       0.85      0.01      0.02      4741

avg / total       0.68      0.50      0.35      9482



In [146]:
def keywords_extraction(query):
    query = query.split(' ')
    query = [removePunc(x) for x in query]
    query = [lemmatize(x.lower()) for x in query]
    q = []
    for w in query:
        if not stopwords.get(w) and w != '':
            q.append(w)
    print(q)
    tagged = nltk.pos_tag(q,tagset="universal")
    return [p[0] for p in tagged if p[1] in ["ADJ","NOUN","VERB","NUM"]]


def relation_extraction(query):
    words = keywords_extraction(query)
    if words == []:
        return
    relations = defaultdict(dict)
    for w1 in words:
        for w2 in words:
            if w1 == w2:
                continue
            else:
                relations[w1][w2] = relations[w1].get(w2, 0) + 1
    return relations

pos = keywords_extraction("a kilogram can be defined as what value")
print(pos)
relations = relation_extraction("a kilogram can be defined as what value?")
print(relations)




def answer_type(query):
    query = query.split()
    query = [removePunc(x) for x in query]
    query = [lemmatize(x.lower()) for x in query]
    for word in query:
        if word == 'what':
            return 'O'
        elif word == 'who':
            return 'PERSON'
        elif word == 'where':
            return 'LOCATION'
        elif word == 'when':
            return 'DATE'
        elif word == 'which':
            return 'O'
    return 'O'
        

['kilogram', 'define', 'what', 'value']
['kilogram', 'define', 'value']
['kilogram', 'define', 'what', 'value']
defaultdict(<class 'dict'>, {'kilogram': {'define': 1, 'value': 1}, 'define': {'kilogram': 1, 'value': 1}, 'value': {'kilogram': 1, 'define': 1}})


In [155]:
def topk_sentence(para,keywords):
    sentences = para.split('.')
    for sent in sentences:
        overlap = 0
        words = keywords_extraction(sent)
        for w in keywords:
            if w in words:
                overlap += 1



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/models/doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


AttributeError: 'str' object has no attribute 'words'